In [8]:
import cv2
import numpy as np
from glob2 import glob
import os, os.path
from platform import platform

n_indexed_colors=256
n_color_histogram_categories=64

In [24]:
im1 = cv2.imread('/Users/Anita/Documents/HMDA/2ndSemester/IREI/CBIR/python_project/testing_images/im1.jpg')
im2 = cv2.imread('/Users/Anita/Documents/HMDA/2ndSemester/IREI/CBIR/python_project/testing_images/im2.jpg')
im3 = cv2.imread('/Users/Anita/Documents/HMDA/2ndSemester/IREI/CBIR/python_project/testing_images/im3.jpg')


# Feature extraction functions

In [10]:
def ColorHistogram(img):
    # Calculte and normalize the histogram for each channel then append them
    histogram = cv2.calcHist([img],[0],None,[n_color_histogram_categories],[0,256])
    histogram = cv2.normalize(histogram, None)

    ch1_histogram = cv2.calcHist([img],[1],None,[n_color_histogram_categories],[0,256])
    ch1_histogram = cv2.normalize(ch1_histogram, None)
    histogram = np.vstack((histogram, ch1_histogram))

    ch2_histogram = cv2.calcHist([img],[2],None,[n_color_histogram_categories],[0,256])
    ch2_histogram = cv2.normalize(ch2_histogram, None)
    histogram = np.vstack((histogram, ch2_histogram))

    return histogram

def TextureGaborFilter(img):
    # returns the img filtered by the filter list of kernels in several orientations
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 30):
        params = {'ksize':(ksize, ksize), 'sigma':1.0, 'theta':theta, 'lambd':15.0,
                  'gamma':0.02, 'psi':0, 'ktype':cv2.CV_32F}
        kern = cv2.getGaborKernel(**params)
        kern /= 1.5*kern.sum()
        filters.append((kern,params))

    accum = np.zeros_like(img)
    for kern,params in filters:
        fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
        np.maximum(accum, fimg, accum)
    accum = accum.reshape(-1, 1)
    return accum

def FusionFeaturesVector(img):
    # Create the features of each category
    features_vector = ColorHistogram(img)

    gabor_filtered = TextureGaborFilter(img)
    features_vector = np.vstack((features_vector, gabor_filtered))

    return features_vector

# Index images and store

In [27]:
#dataPath = os.path.realpath('') + '/testing_images/'
dataPath = os.path.realpath('') + '/data/'
data = glob(dataPath)
imgFiles = []
imgFiles.extend(glob(dataPath + '*.JPG'))
imgFiles.extend(glob(dataPath + '*.JPEG'))
imgFiles.extend(glob(dataPath + '*.BMP'))
imgFiles.extend(glob(dataPath + '*.PNG'))

# windows is case insensitive so we don't need to add this
if not platform().startswith('Windows'):
    imgFiles.extend(glob(dataPath + '*.jpg'))
    imgFiles.extend(glob(dataPath + '*.jpeg'))
    imgFiles.extend(glob(dataPath + '*.bmp'))
    imgFiles.extend(glob(dataPath + '*.png'))

# create the feature vector for every image
for i, imgFile in enumerate(imgFiles):
    # read image file
    img = cv2.imread(imgFile, 1)
    # convert all images to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Save the features vector to files
    histFeaturesVector = ColorHistogram(img)
    texFeaturesVector = TextureGaborFilter(img)
    fusionFeaturesVector = FusionFeaturesVector(img)

    if( imgFile[len(imgFile)-6:] == '.JPEG' or imgFile[len(imgFile)-6:] == '.jpeg' ):
        histFeaturesVectorFileName = dataPath + 'histogramIdx/' + imgFile[70:len(imgFile)-5]
        texFeaturesVectorFileName = dataPath + 'textureIdx/' + imgFile[70:len(imgFile)-5]
        fusionFeaturesVectorFileName = dataPath + 'fusionIdx/' + imgFile[70:len(imgFile)-5]

    else:
        histFeaturesVectorFileName = dataPath + 'histogramIdx/' + imgFile[70:len(imgFile)-4]
        texFeaturesVectorFileName = dataPath + 'textureIdx/' + imgFile[70:len(imgFile)-4]
        fusionFeaturesVectorFileName = dataPath + 'fusionIdx/' + imgFile[70:len(imgFile)-4]

    np.save(histFeaturesVectorFileName, histFeaturesVector)
    np.save(texFeaturesVectorFileName, texFeaturesVector)
    np.save(fusionFeaturesVectorFileName, fusionFeaturesVector)

# Pruebas

In [56]:
h1 = ColorHistogram(im1)
tgf1 = TextureGaborFilter(im1)
fv1 = FusionFeaturesVector(im1)

h1.shape, tgf1.shape, fv1.shape

((192, 1), (150528, 1), (150720, 1))

In [8]:
histogram1 = cv2.calcHist([im1],[0],None,[n_color_histogram_categories],[0,256])
histogram1 = cv2.normalize(histogram1, None)

histogram2 = cv2.calcHist([im2],[0],None,[n_color_histogram_categories],[0,256])
histogram2 = cv2.normalize(histogram2, None)

histogram3 = cv2.calcHist([im3],[0],None,[n_color_histogram_categories],[0,256])
histogram3 = cv2.normalize(histogram3, None)

In [13]:
comp11=cv2.compareHist(histogram1,histogram1,0)
comp12=cv2.compareHist(histogram1,histogram2,0)
comp13=cv2.compareHist(histogram1,histogram3,0)
comp11, comp12, comp13

(1.0, 0.9485386406141665, 0.9777073051278486)